In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import torch
import random

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis, ActivationStoreAnalysis
from unlearning.metrics import modify_and_calculate_metrics, calculate_metrics_list

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np
import pandas as pd
import itertools
from transformer_lens import utils

from jaxtyping import Float
from torch import Tensor

from pathlib import Path

import plotly.express as px
from unlearning.var import REPO_ID, SAE_MAPPING
import pickle

from unlearning.metrics import all_permutations

In [3]:
# Load main SAE for gemma-2b-it
filename = hf_hub_download(repo_id=REPO_ID, filename=SAE_MAPPING['gemma_2b_it_resid_pre_9'])
sae = load_saved_sae(filename)
model = model_store_from_sae(sae)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [4]:
activation_store = ActivationStoreAnalysis(sae.cfg, model)

buffer
dataloader


In [5]:
# load good feature list

with open('../yeutong_notebooks/unlearning_output/good_features_list_v1.pkl', 'rb') as f:
    good_features_list = pickle.load(f)


features_to_test = list(set([item for sublist in good_features_list.values() for item in sublist]))
filtered_good_features = [12663, 4342, 5749, 10355, 1523, 15858, 12273, 14315, 4451, 1611, 10051, 16186, 7983, 6958, 1307, 11019, 6531, 12289]
filtered_features_sorted_by_loss = [7983, 16186, 12273, 14315,  4342, 10051, 15858,  6958, 12663, 1611,  6531,  1523, 10355,  5749,  1307, 12289,  4451, 11019]
filtered_features_sorted_by_loss2 = np.concatenate((filtered_features_sorted_by_loss[:8], filtered_features_sorted_by_loss[10:11], filtered_features_sorted_by_loss[12:]))

unlearning_dataset = ['wmdp-bio']
side_effect_dataset_names =  ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']


In [6]:
main_ablate_params = {
    'features_to_ablate': [],
    'multiplier': 15,
    'intervention_method': 'clamp_feature_activation',
}

sweep = {
    'features_to_ablate': [filtered_features_sorted_by_loss2],
}


question_subset = [ 15,  20,  24,  28,  34,  45,  49,  61,  63,  68,  69,  70,  72, 86,  95,  97,  99, 111, 112, 122, 123, 132, 145, 151, 173, 174, 175, 177, 182, 185, 193]

metric_params = { 'high_school_us_history': 
                 {
                    'target_metric': None,
                    'question_subset': question_subset,
                 }
                }


# Which metrics to get
dataset_names = ['wmdp-bio', 'high_school_us_history'] 
dataset_names = unlearning_dataset + side_effect_dataset_names
# dataset_names = []
compute_loss = True


metrics_list = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      metric_params=metric_params,
                                      dataset_names=dataset_names,
                                      compute_loss=compute_loss,
                                      n_batch_loss_added=50,
                                     activation_store=activation_store)

# metrics_dict = convert_list_of_dicts_to_dict_of_lists(metrics_list)



The following are multiple choice questions (with answers) about biology.



Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

100%|██████████| 29/29 [00:06<00:00,  4.36it/s]


Generating test split:   0%|          | 0/204 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/22 [00:00<?, ? examples/s]

...

100%|██████████| 6/6 [00:00<00:00,  6.40it/s]


Generating test split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 50/50 [01:02<00:00,  1.25s/it]

In [7]:
[metrics_list[0][x]['mean_correct'] for x in dataset_names]

[0.45348837971687317,
 1.0,
 1.0,
 0.9333333969116211,
 0.78125,
 0.6000000238418579]

In [8]:
dataset_names

['wmdp-bio',
 'high_school_us_history',
 'college_computer_science',
 'high_school_geography',
 'human_aging',
 'college_biology']

In [23]:
metrics_list[0]['high_school_us_history']['mean_correct'], metrics_list[0]['wmdp-bio']['mean_correct'], metrics_list[0]['loss_added']

(1.0, 0.3604651093482971, 0.0251005220413208)

In [20]:
metrics_list[0]['high_school_us_history']['mean_correct'], metrics_list[0]['wmdp-bio']['mean_correct'], metrics_list[0]['loss_added']

(1.0, 0.45348837971687317, 0.008123407363891602)

In [22]:
metrics_list[0]['high_school_us_history']['mean_correct'], metrics_list[0]['wmdp-bio']['mean_correct'], metrics_list[0]['loss_added']

(1.0, 0.3604651093482971, 0.0251005220413208)

In [15]:
metrics_list[0]['loss_added']

0.03617446660995483

In [142]:
main_ablate_params = {
    'features_to_ablate': [],
    'multiplier': 20,
    'intervention_method': 'clamp_feature_activation',
}

sweep = {
    'features_to_ablate': [filtered_features_sorted_by_loss2[:i+1] for i in range(1)],
}


question_subset = [ 15,  20,  24,  28,  34,  45,  49,  61,  63,  68,  69,  70,  72, 86,  95,  97,  99, 111, 112, 122, 123, 132, 145, 151, 173, 174, 175, 177, 182, 185, 193]

metric_params = { 'high_school_us_history': 
                 {
                    'target_metric': None,
                     'without_question': False,
                     'question_subset': question_subset,
                     'permutations': None,
                 }
                }


# Which metrics to get
dataset_names = ['wmdp-bio', 'high_school_us_history'][:1]
compute_loss = False




metrics_list = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      metric_params=metric_params,
                                      dataset_names=dataset_names,
                                      compute_loss=compute_loss,
                                      n_batch_loss_added=10,
                                     activation_store=activation_store)

# metrics_dict = convert_list_of_dicts_to_dict_of_lists(metrics_list)



The following are multiple choice questions (with answers) about biology.



100%|██████████| 29/29 [00:06<00:00,  4.26it/s]

In [143]:
[x['wmdp-bio']['mean_correct'] for x in metrics_list], [x['wmdp-bio']['total_correct'] for x in metrics_list]

([0.9767441749572754], [168])

In [141]:
170/172

0.9883720930232558

In [131]:
metrics_dict = convert_list_of_dicts_to_dict_of_lists(metrics_list)


In [132]:
metrics_dict['loss_added']

[-0.0010888338088989257, -0.001949143409729004, -0.001949143409729004]

[0.9767441749572754, 0.9709302186965942, 0.8604651093482971]

In [110]:
metrics_list[0]['high_school_us_history']['is_correct'].reshape(-1, 24).sum(axis=1)

array([24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24.,
       24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24., 24.,
       24.], dtype=float32)

In [117]:
a = metrics_list[0]['high_school_us_history']['is_correct'].reshape(-1, 24).sum(axis=1)
len(a)

31

100%|██████████| 6/6 [00:03<00:00,  1.68it/s]

In [101]:
[x['mean_correct'] for x in metrics_list['high_school_us_history']]

[1.0]

In [89]:
from unlearning.metrics import convert_list_of_dicts_to_dict_of_lists

[0.05323126316070557, 0.05261061191558838]

In [84]:
[x['wmdp-bio']['mean_correct'] for x in metrics_list]

KeyError: 'wmdp-bio'

In [85]:
[x['high_school_us_history']['is_correct'][:-1].mean() for x in metrics_list]

[1.0, 1.0]

In [87]:
[x['loss_added'] for x in metrics_list]

[0.05323126316070557, 0.05261061191558838]

In [26]:
all_permutations_correct_index = [ 15,  20,  24,  28,  34,  45,  49,  61,  63,  68,  69,  70,  72, 86,  95,  97,  99, 111, 112, 122, 123, 132, 145, 151, 173, 174, 175, 177, 182, 185, 193]


In [112]:
history_correct = '../data/mmlu_high_school_us_history_gemma-2b-it_correct.csv'
new_vals = np.genfromtxt(history_correct)
new_vals, len(new_vals)

(array([ 15.,  20.,  24.,  28.,  34.,  45.,  49.,  63.,  68.,  69.,  70.,
         72.,  79.,  97., 111., 112., 123., 132., 145., 151., 173., 174.,
        175., 177., 182., 185., 193.]),
 27)

In [114]:
np.array(all_permutations_correct_index).astype(float).shape

(31,)

In [32]:
history_correct = '../data/mmlu_high_school_us_history_gemma-2b-it_correct_not_correct_wo_question_prompt.csv'
new_vals2 = np.genfromtxt(history_correct)
new_vals2

array([ 15.,  20.,  24.,  28.,  34.,  45.,  49.,  63.,  68.,  69.,  70.,
        72.,  79.,  97., 111., 112., 123., 132., 151., 173., 174., 175.,
       177., 182., 185., 193.])

In [ ]:
[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9000000357627869,
 0.8666667342185974,
 1.0,
 0.46666669845581055,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]